In [1]:
from qonnx.core.modelwrapper import ModelWrapper
import os
onnx_dir = 'onnx_files/'
if not(os.path.exists(onnx_dir)):
    os.mkdir(onnx_dir)
    
model = ModelWrapper("multiply_model.onnx")

In [2]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
model = ModelWrapper("multiply_model.onnx")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(onnx_dir + "multiply_model_finn.onnx")

In [4]:
import finn.transformation.streamline.absorb as absorb
from qonnx.transformation.general import RemoveUnusedTensors
from qonnx.transformation.infer_data_layouts import InferDataLayouts

model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
# bit of tidy-up
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())

model.save(onnx_dir+"/multiply_ready_for_hls_conversion.onnx")

/workspace/qonnx/src/qonnx/transformation/infer_data_layouts.py:119: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [7]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
model = ModelWrapper(onnx_dir+"/multiply_ready_for_hls_conversion.onnx")
model = model.transform(to_hls.InferBinaryMatrixVectorActivation("decoupled"))
# TopK to LabelSelect
model = model.transform(to_hls.InferLabelSelectLayer())
# input quantization (if any) to standalone thresholding
model = model.transform(to_hls.InferThresholdingLayer())
model.save(onnx_dir+"/multiply_hls_layers.onnx")

In [9]:
# change this if you have a different PYNQ board, see list above
from finn.util.basic import pynq_part_map
pynq_board = "KV260_SOM"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [10]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(onnx_dir+"/multiply_hls_layers.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

AttributeError: 'NoneType' object has no attribute 's'